# Deep Learning

In [1]:
import os
import itertools
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
%matplotlib inline

## Loading Data

In [3]:
# ecg_path = "C://Users//HP//Documents//Varun//DHAI//Capstone//checkpoint//processed_data//ecg_processed_data.csv"
ecg_path = "C://Users//HP//Documents//Varun//DHAI//Capstone//checkpoint//processed_data//ecg_processed_data_2.csv"

In [4]:
ecg_data = pd.read_csv(ecg_path, index_col=0)

In [5]:
ecg_data.head()

,Segment Start,Segment End,xs0,xs1,xs2,xs3,xs4,xs5,xs6,xs7,...,xs193,xs194,xs195,xs196,xs197,xs198,xs199,Annotation Class,Annotation Class Numeric,acn
Record ID,,,,,,,,,,,,,,,,,,,,,
100,13,213,0.059449,0.055308,0.052035,0.049666,0.047961,0.046922,0.046801,0.047770,...,0.033233,0.033981,0.034276,0.034166,0.033702,0.033134,0.032648,N,1,1
100,307,507,0.084239,0.087566,0.088956,0.088621,0.086919,0.084459,0.081823,0.079506,...,0.020049,0.020272,0.020719,0.020971,0.020866,0.020622,0.020434,N,1,1
100,563,763,-0.006937,-0.005690,-0.005184,-0.005172,-0.005405,-0.005802,-0.006385,-0.007163,...,-0.084798,-0.084841,-0.084325,-0.082953,-0.080644,-0.077424,-0.073181,N,1,1
100,883,1083,0.072086,0.073128,0.073026,0.071802,0.069731,0.067336,0.065156,0.063624,...,0.040605,0.040177,0.039328,0.038051,0.036671,0.035565,0.034817,N,1,1
100,1168,1368,0.084762,0.083995,0.082327,0.080127,0.077701,0.075216,0.072963,0.071264,...,0.025977,0.027249,0.029211,0.031715,0.034364,0.036462,0.037443,N,1,1


In [6]:
ecg_data["Annotation Class"].unique()

array(['N', 'V', '/', 'L', 'R'], dtype=object)

In [7]:
ecg_data["Annotation Class"].nunique()

5

In [8]:
ecg_data["Annotation Class"].value_counts()

N    73439
L     8068
R     7255
V     6793
/     3619
Name: Annotation Class, dtype: int64

## Preprocessing

### Converting Non Numeric Column to Numeric discontinuous Columns

In [9]:
# annotation_dict = dict()
# for i, symbol in enumerate(ecg_data["Annotation Class"].unique()):
#     annotation_dict[symbol] = i + 1
# ecg_data["Annotation Class Numeric"] = ecg_data["Annotation Class"].apply(
#     lambda x: annotation_dict[x]
# )

In [10]:
ecg_data.head()

,Segment Start,Segment End,xs0,xs1,xs2,xs3,xs4,xs5,xs6,xs7,...,xs193,xs194,xs195,xs196,xs197,xs198,xs199,Annotation Class,Annotation Class Numeric,acn
Record ID,,,,,,,,,,,,,,,,,,,,,
100,13,213,0.059449,0.055308,0.052035,0.049666,0.047961,0.046922,0.046801,0.047770,...,0.033233,0.033981,0.034276,0.034166,0.033702,0.033134,0.032648,N,1,1
100,307,507,0.084239,0.087566,0.088956,0.088621,0.086919,0.084459,0.081823,0.079506,...,0.020049,0.020272,0.020719,0.020971,0.020866,0.020622,0.020434,N,1,1
100,563,763,-0.006937,-0.005690,-0.005184,-0.005172,-0.005405,-0.005802,-0.006385,-0.007163,...,-0.084798,-0.084841,-0.084325,-0.082953,-0.080644,-0.077424,-0.073181,N,1,1
100,883,1083,0.072086,0.073128,0.073026,0.071802,0.069731,0.067336,0.065156,0.063624,...,0.040605,0.040177,0.039328,0.038051,0.036671,0.035565,0.034817,N,1,1
100,1168,1368,0.084762,0.083995,0.082327,0.080127,0.077701,0.075216,0.072963,0.071264,...,0.025977,0.027249,0.029211,0.031715,0.034364,0.036462,0.037443,N,1,1


In [11]:
# rename_col_dict = {str(i): f"xs{i}" for i in range(0, 200)}
# ecg_data = ecg_data.rename(columns=rename_col_dict)
# ecg_data.to_csv("C://Users//HP//Documents//Varun//DHAI//Capstone//checkpoint//processed_data//ecg_processed_data_2.csv", index=False)

In [12]:
# x_columns = [str(i) for i in range(0, 200)]
x_columns = [f"xs{i}" for i in range(0, 200)]

In [13]:
import torch

class Config:
    csv_path = ''
    seed = 2021
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    attn_state_path = '../input/mitbih-with-synthetic/attn.pth'
    lstm_state_path = '../input/mitbih-with-synthetic/lstm.pth'
    cnn_state_path = '../input/mitbih-with-synthetic/cnn.pth'
    
    attn_logs = '../input/mitbih-with-synthetic/attn.csv'
    lstm_logs = '../input/mitbih-with-synthetic/lstm.csv'
    cnn_logs = '../input/mitbih-with-synthetic/cnn.csv'
    
    train_csv_path = '../input/mitbih-with-synthetic/mitbih_with_syntetic_train.csv'
    test_csv_path = '../input/mitbih-with-synthetic/mitbih_with_syntetic_test.csv'

def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
config = Config()
seed_everything(config.seed)

## What each Annotation looks like?

In [ ]:
for symbol in ecg_data["Annotation Class"].unique():
    readings = (
        ecg_data[ecg_data["Annotation Class"] == symbol].head(1)[x_columns].values[0]
    )
    plt.figure(figsize=(10, 8))
    plt.title(label=symbol)
    plt.plot(readings)
    plt.show()

## Data Loader

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class ECGDataset(Dataset):

    def __init__(self, df):
        self.df = df
        self.data_columns = self.df.columns[3:-3].tolist()

    def __getitem__(self, idx):
        signal = self.df.loc[idx, self.data_columns].astype('float32')
        signal = torch.FloatTensor([signal.values])                 
        target = torch.LongTensor(np.array(self.df.loc[idx, 'acn']))
        return signal, target

    def __len__(self):
        return len(self.df)

In [ ]:
class ECGDataset(Dataset):

    def __init__(self, df):
        self.df = df
        self.data_columns = self.df.columns[3:-3].tolist()

    def __getitem__(self, idx):
        signal = self.df[self.data_columns].iloc[idx].astype('float32')
        signal = torch.FloatTensor([signal.values])
        target = torch.LongTensor(self.df['acn'][idx])
        return signal, target

    def __len__(self):
        return len(self.df)

In [ ]:
ds = ECGDataset(ecg_data)
dataloader = DataLoader(dataset=ds, batch_size=6)
for i, (s,t) in enumerate(dataloader):
    print(i, s.shape, t)

In [ ]:
def get_dataloader(train_csv_path: str, phase: str, batch_size: int = 96) -> DataLoader:
    '''
    Dataset and DataLoader.
    Parameters:
        train_csv_path: processed_data path.
        phase: training, testing or validation phase.
        target: Annotation Class Numeric
        signal_cols: columns belonging to signal
        batch_size: data per iteration.
    Returns:
        data generator
    '''
    df = pd.read_csv(train_csv_path)
   
    t_df, test_df = train_test_split( df, test_size=0.2, random_state=42, stratify=df['acn'])
    
    t_df, test_df = t_df.reset_index(drop=True), test_df.reset_index(drop=True)
    
    train_df, val_df = train_test_split(t_df, test_size=0.2, random_state=42, stratify=t_df['acn'])

    train_df, val_df = train_df.reset_index(drop=True), val_df.reset_index(drop=True)

    if phase == 'train':
        df = train_df
    elif phase == 'val':
        df = val_df
    else:
        df = test_df

    dataset = ECGDataset(df)
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=4)
    
    return dataloader

## Trainer and Metrics

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import (CosineAnnealingLR,
                                      CosineAnnealingWarmRestarts,
                                      StepLR,
                                      ExponentialLR)

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, auc, f1_score, precision_score, recall_score


class Meter:
    def __init__(self, n_classes=5):
        self.metrics = {}
        self.confusion = torch.zeros((n_classes, n_classes))

    def update(self, x, y, loss):
        x = np.argmax(x.detach().cpu().numpy(), axis=1)
        y = y.detach().cpu().numpy()
        self.metrics['loss'] += loss
        self.metrics['accuracy'] += accuracy_score(x, y)
        self.metrics['f1'] += f1_score(x, y, average='macro')
        self.metrics['precision'] += precision_score(x, y, average='macro', zero_division=1)
        self.metrics['recall'] += recall_score(x, y, average='macro', zero_division=1)

        self._compute_cm(x, y)

    def _compute_cm(self, x, y):
        for prob, target in zip(x, y):
            if prob == target:
                self.confusion[target][target] += 1
            else:
                self.confusion[target][prob] += 1

    def init_metrics(self):
        self.metrics['loss'] = 0
        self.metrics['accuracy'] = 0
        self.metrics['f1'] = 0
        self.metrics['precision'] = 0
        self.metrics['recall'] = 0

    def get_metrics(self):
        return self.metrics

    def get_confusion_matrix(self):
        return self.confusion


class Trainer:
    def __init__(self, train_csv_path, net, lr, batch_size, num_epochs):
        self.net = net.to(config.device)
        self.num_epochs = num_epochs
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = AdamW(self.net.parameters(), lr=lr)
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=num_epochs, eta_min=5e-6)
        self.best_loss = float('inf')
        self.phases = ['train', 'val']
        self.dataloaders = {
            phase: get_dataloader(train_csv_path, phase, batch_size) for phase in self.phases
        }
        self.train_df_logs = pd.DataFrame()
        self.val_df_logs = pd.DataFrame()

    def _train_epoch(self, phase):
        print(f"{phase} mode | time: {time.strftime('%H:%M:%S')}")

        self.net.train() if phase == 'train' else self.net.eval()
        meter = Meter()
        meter.init_metrics()
        
        print(1)

        for i, (data, target) in enumerate(self.dataloaders[phase]):
            data = data.to(config.device)
            target = target.to(config.device)

            output = self.net(data)
            loss = self.criterion(output, target)

            if phase == 'train':
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            meter.update(output, target, loss.item())
 
            
        metrics = meter.get_metrics()
        metrics = {k: v / i for k, v in metrics.items()}
        df_logs = pd.DataFrame([metrics])
        confusion_matrix = meter.get_confusion_matrix()
        print("completed metrics calc")
        if phase == 'train':
            self.train_df_logs = pd.concat([self.train_df_logs, df_logs], axis=0)
        else:
            self.val_df_logs = pd.concat([self.val_df_logs, df_logs], axis=0)

        # show logs
        print('{}: {}, {}: {}, {}: {}, {}: {}, {}: {}'
              .format(*(x for kv in metrics.items() for x in kv))
              )
        fig, ax = plt.subplots(figsize=(5, 5))
        cm_ = ax.imshow(confusion_matrix, cmap='hot')
        ax.set_title('Confusion matrix', fontsize=15)
        ax.set_xlabel('Actual', fontsize=13)
        ax.set_ylabel('Predicted', fontsize=13)
        plt.colorbar(cm_)
        plt.show()

        return loss

    def run(self):
        for epoch in range(self.num_epochs):
            self._train_epoch(phase='train')
            with torch.no_grad():
                val_loss = self._train_epoch(phase='val')
                self.scheduler.step()

            if val_loss < self.best_loss:
                self.best_loss = val_loss
                print('\nNew checkpoint\n')
                self.best_loss = val_loss
                torch.save(self.net.state_dict(), f"best_model_epoc{epoch}.pth")
            # clear_output()


## Models

In [ ]:
from utils.models import CNN, RNNModel, RNNAttentionModel

In [ ]:
model = CNN(num_classes=5, hid_size=128)

## Experimentation

In [ ]:
trainer = Trainer(train_csv_path=ecg_path , net=model, lr=1e-3, batch_size=96, num_epochs=10)#100)
trainer.run()